In [1]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))


In [2]:
# para actualizar cambios del .py en el IPython notebook
%reload_ext autoreload
%autoreload 2

In [3]:
from data import load_raw_data, load_last_12_months_data

# Descargamos el archivo de datos crudos
# para Enero de 2024
# Cargamos el archivo de datos crudos
# Validamos el archivo con las fechas

df = load_raw_data(2024, [1])

# Mostramos estadísticos
df.describe()

Archivo 2024-01 ya está disponible localmente


,pickup_datetime,pickup_location_id
count,2964606,2.964606e+06
mean,2024-01-17 01:02:08.093335,1.660179e+02
min,2024-01-01 00:00:00,1.000000e+00
25%,2024-01-09 15:59:24,1.320000e+02
50%,2024-01-17 10:45:42,1.620000e+02
75%,2024-01-24 18:23:53,2.340000e+02
max,2024-01-31 23:59:55,2.650000e+02
std,NaN,6.362399e+01


In [4]:
df['pickup_datetime'].describe(include='all')

count                       2964606
mean     2024-01-17 01:02:08.093335
min             2024-01-01 00:00:00
25%             2024-01-09 15:59:24
50%             2024-01-17 10:45:42
75%             2024-01-24 18:23:53
max             2024-01-31 23:59:55
Name: pickup_datetime, dtype: object

In [5]:
from data import transform_to_time_series

# Transformamos el dataframe a una serie temporal
ts_data = transform_to_time_series(df)
ts_data

/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:148: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
Rellenando huecos por localización: 100%|██████████| 260/260 [00:00<00:00, 1123.47it/s]


,pickup_hour,pickup_location_id,rides
0,2024-01-01 00:00:00,4,25
1,2024-01-01 01:00:00,4,29
2,2024-01-01 02:00:00,4,34
3,2024-01-01 03:00:00,4,31
4,2024-01-01 04:00:00,4,32
...,...,...,...
193435,2024-01-31 19:00:00,245,0
193436,2024-01-31 20:00:00,245,0
193437,2024-01-31 21:00:00,245,0
193438,2024-01-31 22:00:00,245,0


In [6]:
ts_data[ts_data.pickup_location_id == 138]

,pickup_hour,pickup_location_id,rides
55800,2024-01-01 00:00:00,138,59
55801,2024-01-01 01:00:00,138,4
55802,2024-01-01 02:00:00,138,0
55803,2024-01-01 03:00:00,138,0
55804,2024-01-01 04:00:00,138,0
...,...,...,...
56539,2024-01-31 19:00:00,138,200
56540,2024-01-31 20:00:00,138,205
56541,2024-01-31 21:00:00,138,137
56542,2024-01-31 22:00:00,138,222


In [7]:
from data import transform_to_features_and_target

# Transformamos la serie temporal a un dataframe de características y objetivo
location_id = 138
n_lags = 24*28 # 1 días

X, y, df_location = transform_to_features_and_target(ts_data, location_id, n_lags)
df_location


/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

,pickup_hour,pickup_location_id,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,...,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,target
0,2024-01-29 00:00:00,138,59.0,4.0,0.0,0.0,0.0,1.0,1.0,8.0,...,207.0,256.0,199.0,243.0,217.0,307.0,251.0,302.0,247.0,138
1,2024-01-29 01:00:00,138,4.0,0.0,0.0,0.0,1.0,1.0,8.0,64.0,...,256.0,199.0,243.0,217.0,307.0,251.0,302.0,247.0,138.0,15
2,2024-01-29 02:00:00,138,0.0,0.0,0.0,1.0,1.0,8.0,64.0,139.0,...,199.0,243.0,217.0,307.0,251.0,302.0,247.0,138.0,15.0,25
3,2024-01-29 03:00:00,138,0.0,0.0,1.0,1.0,8.0,64.0,139.0,199.0,...,243.0,217.0,307.0,251.0,302.0,247.0,138.0,15.0,25.0,2
4,2024-01-29 04:00:00,138,0.0,1.0,1.0,8.0,64.0,139.0,199.0,138.0,...,217.0,307.0,251.0,302.0,247.0,138.0,15.0,25.0,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2024-01-31 19:00:00,138,238.0,151.0,95.0,116.0,150.0,80.0,12.0,22.0,...,152.0,132.0,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200
68,2024-01-31 20:00:00,138,151.0,95.0,116.0,150.0,80.0,12.0,22.0,4.0,...,132.0,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205
69,2024-01-31 21:00:00,138,95.0,116.0,150.0,80.0,12.0,22.0,4.0,0.0,...,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205.0,137
70,2024-01-31 22:00:00,138,116.0,150.0,80.0,12.0,22.0,4.0,0.0,4.0,...,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205.0,137.0,222


In [8]:
# guardamos datos
from paths import TRANSFORMED_DATA_DIR

df_location.to_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_1year_4weeks_lags.parquet')